In [1]:
!nvidia-smi

Sun Dec 13 07:24:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install bert-for-tf2 >> /dev/null

In [3]:
import os
import math
import datetime
import importlib

# from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm

import pandas as pd
import numpy as np

import re

import tensorflow as tf
from tensorflow import keras
from  tensorflow.keras.callbacks import EarlyStopping

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

import random

In [4]:
cd "/content/drive/My Drive/Colab Notebooks NU"

/content/drive/.shortcut-targets-by-id/1FqifPUlm6cgaaWF0H2AQqipYJL1XcCct/Colab Notebooks


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import bert_thesis_experiments.utils as utils
import bert_thesis_experiments.my_models as my_models

In [7]:
seqfile = 'DeepDom_Code/DeepDom-master/processed_seq.txt'; #file name of the processed sequence data (output from dataprocess.pl)
labelfile= 'DeepDom_Code/DeepDom-master/processed_label.txt'; #file name of the processed label data (output from dataprocess.pl)

In [8]:
(ids,seqs) = utils.process_inputseqs(seqfile)
(ids,labels) = utils.process_inputlabels(labelfile)

In [9]:
# num_of_seq = 20000
# rawdata = list(zip(seqs[:num_of_seq],labels[:num_of_seq]))
# random.shuffle(rawdata)

rawdata = list(zip(seqs,labels))
random.shuffle(rawdata)

In [10]:
tokenizer = FullTokenizer(vocab_file=os.path.join("bert_thesis_experiments/working/", "protein_seq_words_uniref50-vocab.txt"))

In [11]:
seq_i = 51
print(rawdata[seq_i][0])
print("Number of tokens in seq", len(tokenizer.tokenize(rawdata[seq_i][0])))
print(tokenizer.tokenize(rawdata[seq_i][0]))

QDAWNFVAPLPNPLAEFSACECKGKIYVIGGYTTRDRNMNILQYCPSSDMWTLFETCDVHIRKQQMVSVEETIYIVGGCLHELGPNRRSSQSEDMLTVQSYNTVTRQWLYLKENTSKSGLNLTCALHNDGIYIMSRDVTLSTSLEHRVFLKYNIFSDSWEAFRRFPAFGHNLLVSSLYLPNKAET---------------
Number of tokens in seq 80
['q', '##d', '##awn', '##fvap', '##l', '##pnpl', '##aef', '##sac', '##eck', '##gk', '##iy', '##vigg', '##yt', '##tr', '##dr', '##nm', '##nil', '##qyc', '##p', '##ssd', '##mw', '##tlfe', '##tc', '##dvhi', '##rkqq', '##mvsv', '##eeti', '##yi', '##vg', '##g', '##clh', '##elgp', '##nr', '##rssq', '##sedm', '##l', '##tvq', '##synt', '##vtr', '##qw', '##l', '##ylken', '##ts', '##ksgl', '##nltc', '##alhn', '##dgiy', '##im', '##srdv', '##tlst', '##sleh', '##rvfl', '##kyni', '##fsd', '##sw', '##eaf', '##rrfp', '##afgh', '##nll', '##v', '##ssly', '##l', '##pn', '##kae', '##t', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]']


In [12]:
tokens = tokenizer.tokenize(rawdata[seq_i][0])
tokens = ["[CLS]"] + tokens + ["[SEP]"]
token_ids = tokenizer.convert_tokens_to_ids(tokens)
padded_token_ids = token_ids + [0] * (200 - len(token_ids))
print(len(padded_token_ids), padded_token_ids, sep='\n')

200
[2, 20, 34, 3403, 18729, 36, 54703, 1581, 1354, 4690, 285, 332, 19424, 210, 266, 272, 441, 5345, 11605, 47, 512, 1053, 70426, 437, 48180, 14578, 25498, 8657, 203, 166, 43, 3130, 18041, 275, 22584, 29498, 36, 1437, 32819, 27352, 627, 36, 77116, 307, 9857, 33027, 48852, 24832, 425, 12906, 5516, 40843, 11394, 17999, 12074, 218, 866, 29295, 37391, 681, 32, 6252, 36, 327, 1153, 41, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
# importlib.reload(utils)
# importlib.reload(my_models)

In [14]:
# max_seq_len = utils.find_max_seq_len(rawdata, tokenizer)

# print("\nMax Seq Length:", max_seq_len)

In [15]:
print(rawdata[seq_i][1])
print(rawdata[900][1])
print(rawdata[500][1])

00000000000000000000000000000000000000011111111111111111111111111111111111111111111100000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111111---------------
111111111111111111111111111111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000-----------------------------------------------------------------------
11111111111111111111111111111111111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111111111111111111110000000


In [16]:

label_array_1 = list(map(int, rawdata[900][1].replace('-', '0')))
print(label_array_1)
print(len(label_array_1))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
200


In [17]:
inputX = [utils.convert_seq_ids(i[0], tokenizer, 200) for i in rawdata]
inputY = [utils.convertlabels_to_categorical(i[1]) for i in rawdata]
# inputY = [utils.convertlabels_to_binary(i[1]) for i in rawdata]

In [18]:
print("Max length of sequence:", len(inputX[seq_i]))
print("No of sequences", len(inputX))
print(inputX[seq_i])

Max length of sequence: 200
No of sequences 31400
[2, 20, 34, 3403, 18729, 36, 54703, 1581, 1354, 4690, 285, 332, 19424, 210, 266, 272, 441, 5345, 11605, 47, 512, 1053, 70426, 437, 48180, 14578, 25498, 8657, 203, 166, 43, 3130, 18041, 275, 22584, 29498, 36, 1437, 32819, 27352, 627, 36, 77116, 307, 9857, 33027, 48852, 24832, 425, 12906, 5516, 40843, 11394, 17999, 12074, 218, 866, 29295, 37391, 681, 32, 6252, 36, 327, 1153, 41, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [19]:
data=list(zip(inputX,inputY))
random.seed(4)
random.shuffle(data)
random.shuffle(data)
    
train_num=int(len(inputX)*0.9)
train=data[0:train_num]
val=data[train_num:]

trainX=np.array([i[0] for i in train])
trainY=np.array([i[1] for i in train])
# xx=np.dstack(trainX)
# xx=np.rollaxis(xx,-1)

print(trainX.shape)   
# print(trainY.shape) 

yy=np.dstack(trainY)
yy=np.rollaxis(yy,-1)

print(yy.shape)
    
valX=np.array([i[0] for i in val])
valY=np.array([i[1] for i in val])
# valX=np.dstack(valX)
# valX=np.rollaxis(valX,-1)

valY=np.dstack(valY)
valY=np.rollaxis(valY,-1)

vv=(valX,valY)

print("\n---Validation Data---")
print(valX.shape)
print(valY.shape)

(28260, 200)
(28260, 200, 3)

---Validation Data---
(3140, 200)
(3140, 200, 3)


In [20]:
bert_model_name = "bert_H512_A8_uniref50_vocab80k_steps10k"

bert_ckpt_dir = os.path.join("bert_thesis_experiments/working/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "model.ckpt-10000")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

In [21]:
def create_model(max_seq_len, bert_ckpt_file):

    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = None
        bert = BertModelLayer.from_params(bert_params, name="bert")
    
    input_ids = keras.layers.Input(
        shape=(max_seq_len, ),
        dtype='int32',
        name="input_ids"
    )
    bert_output = bert(input_ids)
    
    print("bert shape", bert_output.shape)

    classes = 200

    cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    cls_out = keras.layers.Dropout(0.5)(cls_out)
    logits = keras.layers.Dense(units=512, activation="tanh")(cls_out)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(
        units=classes,
        activation="sigmoid"
    )(logits)

    model = keras.Model(inputs=input_ids, outputs=logits)
    model.build(input_shape=(None, max_seq_len))

    load_stock_weights(bert, bert_ckpt_file)

    return model

In [22]:
def create_model_bilstm(max_seq_len, bert_ckpt_file):

  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = None
      bert = BertModelLayer.from_params(bert_params, name="bert")
        
  input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
  bert_output = bert(input_ids)

  print("bert shape", bert_output.shape)

  classes = 200

  x1 = keras.layers.Bidirectional(keras.layers.LSTM(100,return_sequences=True),merge_mode='sum')(bert_output)
#   x4 = keras.layers.Bidirectional(keras.layers.LSTM(3,return_sequences=True),merge_mode='sum')(bert_output)
    
  x2 = keras.layers.Bidirectional(keras.layers.LSTM(100,return_sequences=True),merge_mode='sum')(x1)

  x3 = keras.layers.Bidirectional(keras.layers.LSTM(100,return_sequences=True),merge_mode='sum')(x2)
  x4 = keras.layers.Bidirectional(keras.layers.LSTM(3,return_sequences=True),merge_mode='sum')(x3)

  output = keras.layers.Activation('softmax')(x4)


#   cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
#   cls_out = keras.layers.Dropout(0.5)(cls_out)
#   logits = keras.layers.Dense(units=512, activation="tanh")(cls_out)
#   logits = keras.layers.Dropout(0.5)(logits)
#   logits = keras.layers.Dense(units=classes, activation="sigmoid")(logits)

  model = keras.Model(inputs=input_ids, outputs=output)
  model.build(input_shape=(None, max_seq_len))

  load_stock_weights(bert, bert_ckpt_file)
        
  return model

In [23]:
# model = my_models.create_model_dense_dropout(200, bert_ckpt_file, bert_config_file)
model = my_models.create_model_bilstm(200, bert_ckpt_file, bert_config_file)

bert shape (None, 200, 512)
Done loading 196 BERT weights from: bert_thesis_experiments/working/bert_H512_A8_uniref50_vocab80k_steps10k/model.ckpt-10000 into <bert.model.BertModelLayer object at 0x7f14914df550> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/LayerNorm/beta/adam_m
	bert/embeddings/LayerNorm/beta/adam_v
	bert/embeddings/LayerNorm/gamma/adam_m
	bert/embeddings/LayerNorm/gamma/adam_v
	bert/embeddings/position_embeddings/adam_m
	bert/embeddings/position_embeddings/adam_v
	bert/embeddings/token_type_embeddings
	bert/embeddings/token_type_embeddings/adam_m
	bert/embeddings/token_type_embeddings/adam_v
	bert/embeddings/word_embeddings/adam_m
	bert/embeddings/word_embeddings/adam_v
	bert/encoder/layer_0/attention/output/LayerNorm/beta/adam_m
	bert/encoder/layer_0/attention/output/LayerNorm/beta/adam_v
	bert/encoder/layer_0/attention/output/LayerNorm/gamma/adam_m
	

In [24]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 200)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 200, 512)          79051776  
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 100)          490400    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 100)          160800    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200, 100)          160800    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200, 3)            2496      
_________________________________________________________________
activation (Activation)      (None, 200, 3)           

In [25]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), optimizer=keras.optimizers.Adam(learning_rate=2e-5), 31
#               metrics=[keras.metrics.BinaryAccuracy(threshold=0.5)]) # dense_ff

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=2e-5), metrics=['accuracy']) # bilstm

# fitHistory_batch = model.fit(x=trainX,y=trainY, batch_size=64, validation_data=vv, epochs=50, callbacks=[early_stopping])
fitHistory_batch = model.fit(x=trainX,y=yy, batch_size=64, validation_data=vv, epochs=20, callbacks=[early_stopping]) # bilstm

Epoch 1/20
442/442 [==============================] - 870s 2s/step - loss: 0.3426 - accuracy: 0.8098 - val_loss: 0.3216 - val_accuracy: 0.8173
Epoch 2/20
442/442 [==============================] - 869s 2s/step - loss: 0.3170 - accuracy: 0.8206 - val_loss: 0.3171 - val_accuracy: 0.8193
Epoch 3/20
442/442 [==============================] - 869s 2s/step - loss: 0.3131 - accuracy: 0.8227 - val_loss: 0.3193 - val_accuracy: 0.8167
Epoch 4/20
442/442 [==============================] - 867s 2s/step - loss: 0.3111 - accuracy: 0.8236 - val_loss: 0.3122 - val_accuracy: 0.8216
Epoch 5/20
442/442 [==============================] - 868s 2s/step - loss: 0.3097 - accuracy: 0.8243 - val_loss: 0.3113 - val_accuracy: 0.8221
Epoch 6/20
442/442 [==============================] - 868s 2s/step - loss: 0.3094 - accuracy: 0.8243 - val_loss: 0.3120 - val_accuracy: 0.8216
Epoch 7/20
442/442 [==============================] - 867s 2s/step - loss: 0.3087 - accuracy: 0.8248 - val_loss: 0.3172 - val_accuracy: 0.8193

In [26]:
model.save("bert-512-bilstm_stacked_lr2e5_uniref_stp10k_voc80k")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: bert-512-bilstm_stacked_lr2e5_uniref_stp10k_voc80k/assets


In [27]:
# !cp bert_thesis_experiments/working/ibert_config.json bert_thesis_experiments/working/bert_H512_A8_uniref50_vocab80k_steps10k